In [96]:
from hilo_mpc import NMPC, Model
from casadi import *
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [97]:
model = Model()

# equations = """
# # Constants
# M = 2.7
# m = 0.833
# l = 0.5
# h = 0.5
# g = 9.81

# # DAE
# d/dt(p(t)) = dp(t)
# d/dt(th(t)) = dth(t)
# d/dt(dp(t)) = 1/(M + m - 3/4*m*cos(th(t))^2) * (3/4*m*g*sin(th(t))*cos(th(t)) ...
# - 1/2*m*l*sin(th(t))*dth(t)^2 + F(k))
# d/dt(dth(t)) = 3/(2*l) * (d/dt(dp(t))*cos(th(t)) + g*sin(th(t)))
# 0 = th(t) - y(t)
# """



equations = """
# Constants
Mc = 2.7
Mp = 0.85955
l = 0.5
g = 9.81
J = 0.076
b = 0.1

# DAE
# d/dt(p(t)) = dp(t)
# d/dt(th(t)) = dth(t)
# d/dt(dp(t)) = (F(k)*J + F(k)*Mp*l**2 + J*Mp*dth(t)**2*l*sin((th(t))) - J*b*dp(t) + Mp**2*(dth(t))**2*l**3*sin((th(t))) - Mp**2*g*l**2*sin(2*(th(t)))/2 - Mp*b*dp(t)*l**2)/(J*Mc + J*Mp + Mc*Mp*l**2 + Mp**2*l**2*sin((th(t)))**2)
# d/dt(dth(t)) = Mp*l*(-F(k)*cos(th(t)) + Mc*g*sin(th(t)) - Mp*dth(t)**2*l*sin(2*th(t))/2 + Mp*g*sin(th(t)) + b*dp(t)*cos(th(t)))/(J*Mc + J*Mp + Mc*Mp*l**2 + Mp**2*l**2*sin(th(t))**2)
# 0 = th(t) - y(t)

d/dt(p(t)) = dp(t)
d/dt(th(t)) = dth(t)
d/dt(dp(t)) = (F(k)*J + F(k)*Mp*l**2 + dth(t)**2*l*sin((th(t))) - dp(t) + (dth(t))**2*l**3*sin((th(t))) - Mp**2*g*l**2*sin(2*(th(t)))/2 - dp(t)*l**2)/( J*Mp + l**2 + 2*sin((th(t)))**2)
d/dt(dth(t)) = (-F(k)*cos(th(t)) + Mc*g*sin(th(t)) - dth(t)**2*l*sin(2*th(t))/2 + Mp*g*sin(th(t)) + dp(t)*cos(th(t)))/( J*Mp + Mc*Mp*l**2 + Mp**2*l**2*sin(th(t))**2)
0 = th(t) - y(t)
"""
model.set_equations(equations)

Solver 'cvodes' is not suitable for DAE systems. Switching to 'idas'...


In [98]:
# Initial conditions
x0 = [0, 0.5, 0, 0]

# Initial guess algebraic states
# z0 = h + l * ca.cos(x0[1]) - h
z0 = x0[1]
#Initial guess input
u0 = 0.0
# Setup the model
dt = .05
model.setup(dt=dt)


nmpc = NMPC(model)

nmpc.quad_stage_cost.add_states(names=['p', 'th'], ref=[0, 0], weights=[100, 10000])

nmpc.quad_stage_cost.add_inputs(names='F', weights=0.01)

nmpc.horizon = 25

nmpc.set_box_constraints(x_ub=[1, 15.8, inf, inf], x_lb=[-1, -15.8, -inf, -inf])

nmpc.set_initial_guess(x_guess=x0, u_guess=u0)

nmpc.setup(options={'print_level': 0})

n_steps = 2000

model.set_initial_conditions(x0=x0, z0=z0)

sol = model.solution
model.dynamical_equations

SX(@1=0.25, @2=0.5, @3=2, [dp, dth, ((((((((0.076*F)+(@1*(0.85955*F)))+((@2*sq(dth))*sin(th)))-dp)+((0.125*sq(dth))*sin(th)))-((1.81197*sin((@3*th)))/@3))-(@1*dp))/(0.315326+(@3*sq(sin(th))))), ((((((26.487*sin(th))-(F*cos(th)))-(((@2*sq(dth))*sin((@3*th)))/@3))+(8.43219*sin(th)))+(dp*cos(th)))/(0.645522+(0.184707*sq(sin(th)))))])

In [99]:
for step in range(n_steps):
    u = nmpc.optimize(x0)
    model.simulate(u=u, steps=1)
    x0 = sol['x:f']

In [100]:
output_notebook()
p_tot = []
for state in model.dynamical_state_names:
    p = figure(background_fill_color="#fafafa", width=300, height=300)
    p.line(x=np.array(sol['t']).squeeze(), y=np.array(sol[state]).squeeze(),
           legend_label=state, line_width=2)
    for i in range(len(nmpc.quad_stage_cost._references_list)):
        if state in nmpc.quad_stage_cost._references_list[i]['names']:
            position = nmpc.quad_stage_cost._references_list[i]['names'].index(state)
            value = nmpc.quad_stage_cost._references_list[i]['ref'][position]
            p.line([np.array(sol['t'][1]).squeeze(), np.array(sol['t'][-1]).squeeze()],
                   [value, value], legend_label=state + '_ref',
                   line_dash='dashed', line_color="red", line_width=2)

    p.yaxis.axis_label = state
    p.xaxis.axis_label = 'time'
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"

    p_tot.append(p)

show(gridplot(p_tot, ncols=2))

Loading BokehJS ...